In [1]:
import os
os.chdir("../")
%pwd

'd:\\PracticeProjects\\NL_Text_Summarization\\NL_TEXT_SUMMARIZER'

In [2]:
# This block will later be moved to entity

from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen = True)
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: Path
    num_train_epochs : int
    warmup_steps : int
    per_device_train_batch_size : int
    per_device_eval_batch_size : int
    weight_decay : float
    logging_steps : int
    evaluation_strategy : str
    eval_steps : int
    save_steps : float
    gradient_accumulation_steps : int   

In [3]:
# Define Config Manager
from NLPTextSummarizer.constants import *
from NLPTextSummarizer.utils.common import read_yaml, create_directories

# 4. Update the configuration manager(./src/project_name/config/configuration.py)
class ConfigurationManager:
    def __init__(self, config_filepath = CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH):
        #print(CONFIG_FILE_PATH)
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath) 
        create_directories([self.config.artifacts_root])

    
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        # Move config path dict from config.yaml
        config = self.config.model_trainer
        params = self.params.TrainingArguments
        # create the directories as mentioned in config.yaml
        create_directories([config.root_dir])
        model_trainer_config = ModelTrainerConfig(
            root_dir = config.root_dir,
            data_path = config.data_path,
            model_ckpt = config.model_ckpt,
            num_train_epochs =  params.num_train_epochs,
            warmup_steps = params.warmup_steps,
            per_device_train_batch_size = params.per_device_train_batch_size,
            per_device_eval_batch_size = params.per_device_eval_batch_size,
            weight_decay = params.weight_decay,
            logging_steps = params.logging_steps,
            evaluation_strategy = params.evaluation_strategy,
            eval_steps = params.eval_steps,
            save_steps = params.save_steps,
            gradient_accumulation_steps = params.gradient_accumulation_steps
        )
        return model_trainer_config

In [4]:
# Define components
# 5. Update the components (./src/project_name/components/__init__.py)
import os
from NLPTextSummarizer.logging import logger
from transformers import DataCollatorForSeq2Seq
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Trainer, TrainingArguments
from datasets import load_from_disk # type: ignore
import torch

class ModelTrainer:
    def __init__(self, config:ModelTrainerConfig):
        self.config = config
    
    def train(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        #tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
        #model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt).to(device)
        #seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model = model_pegasus)

        #load the transformed data
        dataset_samsum_pt = load_from_disk(self.config.data_path)

        # prepare the params for training

        training_args = TrainingArguments(
            output_dir = self.config.root_dir,
            num_train_epochs = self.config.num_train_epochs,
            warmup_steps = self.config.warmup_steps,
            per_device_train_batch_size = self.config.per_device_train_batch_size,
            per_device_eval_batch_size = self.config.per_device_eval_batch_size,
            weight_decay = self.config.weight_decay,
            logging_steps = self.config.logging_steps,
            evaluation_strategy = self.config.evaluation_strategy,
            eval_steps = self.config.eval_steps,
            save_steps = self.config.save_steps,
            gradient_accumulation_steps = self.config.gradient_accumulation_steps            
        )

        # prepare the trainer

        trainer = Trainer(model = model_pegasus,
                          args = training_args,
                          data_collator = seq2seq_data_collator,
                          train_dataset = dataset_samsum_pt["test"],
                          eval_dataset = dataset_samsum_pt["validation"]
                          )
        
        #trainer.train()
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained("transformersbook/pegasus-samsum")
        tokenizer = AutoTokenizer.from_pretrained("transformersbook/pegasus-samsum")
        seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model = model_pegasus)
        # save model
        model_pegasus.save_pretrained(os.path.join(self.config.root_dir,"pegasus-samsum-model"))
        # # save tokenizer
        tokenizer.save_pretrained(os.path.join(self.config.root_dir,"tokenizer"))


d:\Softwares\Anaconda3\envs\NLP_TextSummarization_VENV\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2024-05-08 00:00:35,252: INFO: config: PyTorch version 2.3.0+cu118 available.]


In [5]:
# Pipeline component
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer = ModelTrainer(config = model_trainer_config) 
    model_trainer.train()
except Exception as e:
    raise e 

[2024-05-08 00:00:35,515: INFO: common: yaml file: D:\PracticeProjects\NL_Text_Summarization\NL_TEXT_SUMMARIZER\config\config.yaml loaded successfully]
[2024-05-08 00:00:35,518: INFO: common: yaml file: D:\PracticeProjects\NL_Text_Summarization\NL_TEXT_SUMMARIZER\params.yaml loaded successfully]
[2024-05-08 00:00:35,519: INFO: common: created directory at: artifacts]
[2024-05-08 00:00:35,521: INFO: common: created directory at: artifacts/model_trainer]


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


KeyboardInterrupt: 